# Experiment Tracking

This notebook provides a place to run model experiments and log these experiments to the Vertex AI Experiment Tracker.  

Through Vertex AI Experiments, this notebook will:
1. track the steps of an experiment run (e.g. data ingestion, preprocessing, training)
2. track the inputs to a model (e.g. parameters, datasets)
3. track the outputs from a model run (e.g. models themselves, model metrics, processed datasets)

[Documentation for the Vertex AI SDK](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.ExperimentRun) that we will be using.  
[Documentation around experiment tracking](https://cloud.google.com/vertex-ai/docs/experiments/intro-vertex-ai-experiments) for more background information.
____________________________________________________________________________________________________________

### Steps to Use this Notebook

**Throughout the notebook, each `TODO` comment indicates steps that need to be updated**

1. Declare the constants in the first two blocks
2. Update the model training cells under the 'Perform Model Training' block
3. Run the notebook end to end (run all cells)
4. Check the outcome in the Vertex Experiments UI
    - navigate to the 'Vertex AI' dashboard
    - under the 'Model Development' tab on the left hand side, click on 'Experiments'
    - the experiment, with the name set below, will be shown here

____________________________________________________________________________________________________________________

### Outcomes of running the notebook

1. Create a new experiment, if one isn't already created
    - using the same experiment name across notebook runs will append new experiment runs to the same experiment
2. Create a new run under the experiment name defined in the cell below 
3. Logging 3 artifacts to the experiment run
    - log custom defined parameters (defined in the `PARAMETERS` dictionary)
    - log custom defined metrics (defined in the `METRICS` dictionary)
    - log/upload the trained model to GCS

_____________________

### 1. Declare Experiment Constants

Let's set the constants needed for creating an experiment.

The first box will need to be **updated just once**, at the beginning of each experiment. These are experiment level constants.

The second box will need to be **updated multiple times**, with an update for each run of the notebook/experiment. These are run level constants, which will impact runs inside an experiment.

In [ ]:
# TODO: update these variables when first initiating an experiment

# the name of the experiment that will be used to track our metrics
EXPERIMENT_NAME='forecasting-experiment'
# description of the experiment above
EXPERIMENT_DESCRIPTION='experimenting with model types for forecasting'
# our GCP project ID 
PROJECT_ID=""
# the region of our gcp project
LOCATION="us-central1"

In [ ]:
# TODO: update these variables with each run of an experiment
import time

# the name of this run of the experiment
RUN_NAME='run-{}'.format(int(time.time()))
# the name of our model
MODEL_NAME = "example-model-1"

### 2. Create our experiment

Now that the constants used for our experiment are recorded above, the experiment can be created. There is no code that needs to be updated here.

In [ ]:
from google.cloud import aiplatform

# Create an experiment in our project with the given name & description
def create_experiment_run_sample() -> aiplatform.ExperimentRun:
    # initialise our experiment here using the constants we declared above
    aiplatform.init(
        experiment=EXPERIMENT_NAME,
        experiment_description=EXPERIMENT_DESCRIPTION,
        project=PROJECT_ID,
        location=LOCATION,
    )

    # within our experiment, start a new run for this session
    my_run = aiplatform.start_run(run=RUN_NAME)
    return my_run

gcp_run = create_experiment_run_sample()

### 3. Perform model training

Steps for model training:
1. Import relevant libraries
2. Declare relevant parameters
3. Import training data
4. Train the model

In [ ]:
# TODO: import the relevant libraries here
import pandas as pd
from google.cloud import bigquery

#### Declare our parameters used for training

For each run of this experiment:
1. Declare parameters for the current run
2. Define a dictionary of these parameters for logging purposes

In [ ]:
# TODO: declare model parameters here

# model parameters
MODEL_TYPE='ARIMA_PLUS'
TIMESTAMP_COL = 'receipt_date'
SERIES_ID_COL = 'store_id'
HOLIDAY_REGION = 'NZ'
MODEL_REGISTRY='vertex_ai'

# data parameters
TRAINING_DATA_MIN_DATE = '2017-01-01'
TRAINING_DATA_MAX_DATE = '2022-04-04'
# the percentage of data to be used as training data
TRAIN_SPLIT = 0.8

The `PARAMETERS` dictionary is created for logging purposes. This dictionary will later be logged to our experiment run, with all of the parameters we've defined. This dictionary can have any arbitrary `[key,value]` pair to be logged.

For example, a custom parameter which is not used otherwise can be added as shown below. This is useful when you want to log information to the experiment run.
```
PARAMETERS = {
    'model_type': "XG BOOST", 
    'stores_used': "Only the top 50 stores",
    'notes': "Running the same hyperparameters as before",
}
```

In [ ]:
# TODO: update this dictionary with parameters used
PARAMETERS = {
    'model_type': MODEL_TYPE, 
    'timestamp_col': TIMESTAMP_COL, 
    'series_id_col': SERIES_ID_COL,
    'holiday_region': HOLIDAY_REGION,
    'model_registry': MODEL_REGISTRY,
    'training_date_min_date': TRAINING_DATA_MIN_DATE,
    'training_date_max_date': TRAINING_DATA_MAX_DATE,
}

### Import training data

These cells may or may not need updating for each run of the experiment. Steps include:

1. Helper function to execute queries in BigQuery
2. Defining the SQL query that fetches our training data
3. Executing the above two functions and generate training/validation splits

In [ ]:
# defining the client and function for executing a sql statement on BigQuery

def execute_bq_sql(bq_sql: str) -> pd.DataFrame:
    bq_client = bigquery.Client(project=PROJECT_ID)
    query_job = bq_client.query(bq_sql)
    return query_job.result().to_dataframe()

In [ ]:
# TODO: import training data - the below query is an example query

def get_train_data() -> str:
    train_data_sql = f'''SELECT
       * from training_data
    '''
    return train_data_sql 

In [ ]:
# Extract the data from Bigquery
training_sql = get_train_data()
query_result = execute_bq_sql(training_sql)
# Generate splits
train=query_result.sample(frac=TRAIN_SPLIT,random_state=5)
test=query_result.drop(train.index)

#### Train our model

This cell is for training a custom python model with the data we fetched above.

In [ ]:
from sklearn.svm import SVC

# TODO: perform custom training here & update types
def train_model(train: pd.DataFrame) -> SVC:
    model = SVC()

    return model

model_artifact = train_model(train)

#### Evaluate the model & generate metrics

In this cell, we will extract metrics from our trained model. This requires
1. Generating metrics using the trained model & the test dataset
2. Defining a metrics dictionary that maps the metrics we want to capture to the values of these metrics for this run of the experiment

Note, third party packages are often used to generate these metrics. For example:

```
import sklearn
y_true = test_data['target_column']
y_pred = trained_model.predict(train_data)
mse = sklearn.metrics.mean_squared_error(y_true, y_pred, squared=True)
```

________________

This dictionary is similar to the `PARAMETERS` dictionary, where any `[key,value]` pairs can be added for logging purposes. The user running the notebook needs to decide which metrics are most relevant to log to the experiment run.

Alternative `METRICS` example:
```
METRICS = {'r2_score': .45479, 'MSE': 2342245, 'rmse': 2523, 'quartile': "Tenth"}
```
_______________________

In [ ]:
# TODO: perform model evaluation here
METRICS = {'mse': .9, 'recall': .86}

### 4. Log the outputs of the run

After having trained a model, log the parameters used & training metrics from the model run.

Specifically, we are interestred in capturing:
1. data we are using for each run of a model
2. model artefacts (the trained model itself)
3. metrics of model performance
4. hyperparameters we used to train this model

In [ ]:
# TODO: log additional info, outside of what is already captured here (e.g. custom artifacts)
def log_run(my_run: aiplatform.ExperimentRun, model_artifact: "ExperimentModel") -> None:
    """
    Log the results of the experiment using all of the output recorded above.
    """
    my_run.log_params(PARAMETERS)
    my_run.log_metrics(METRICS)
    my_run.log_model(
        model=model_artifact,
        uri="artifacts-bucket/models/" + MODEL_NAME + "/" + RUN_NAME, 
        display_name=MODEL_NAME,
    )

log_run(gcp_run, model_artifact)


### 5. End our experiment run

We have now created a run of an experiment and logged the key metrics & parameters to our centralised experiment tracking workspace.

Running the following cell will end the run of the experiment, and change the status of the experiment run to completed.

In [ ]:
# end the run of this experiment
gcp_run.end_run()